In [7]:
import requests
import pandas as pd
from tqdm import tqdm


from datetime import datetime
import logging
dateTimeObj = datetime.now()
timestampStr = dateTimeObj.strftime("%Y%m%d%H%M%S")
logging.basicConfig(filename=f'scrapping_logs/run_betexplorer_scrapping_{timestampStr}.log',  level=logging.DEBUG,
    format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p')
timestampStr

'20221209163029'

In [8]:
logging.info('INITIALISING ....')

In [9]:
import bs4 as BeautifulSoup
from bs4 import BeautifulSoup

In [10]:

wcs=[]
for y in range(1999,2023):
    #for m in range(1,10):
    #u = f"https://www.betexplorer.com/soccer/world/friendly-international-{y}/results/?month={y}-0{m}"
    u = f"https://www.betexplorer.com/soccer/england/premier-league-{y-1}-{y}/results/"
    wcs.append(u)
    u = f"https://www.betexplorer.com/soccer/england/championship-{y-1}-{y}/results/"
    wcs.append(u)
    u = f"https://www.betexplorer.com/soccer/england/league-one-{y-1}-{y}/results/"
    wcs.append(u)
    u = f"https://www.betexplorer.com/soccer/france/ligue-1-{y-1}-{y}/results/"
    wcs.append(u)
    u = f"https://www.betexplorer.com/soccer/france/ligue-2-{y-1}-{y}/results/"
    wcs.append(u)
    u = f"https://www.betexplorer.com/soccer/germany/bundesliga-{y-1}-{y}/results/"
    wcs.append(u)
    u = f"https://www.betexplorer.com/soccer/italy/serie-b-{y-1}-{y}/results/"
    wcs.append(u)
    u = f"https://www.betexplorer.com/soccer/italy/serie-a-{y-1}-{y}/results/"
    wcs.append(u)



#wcs= ['https://www.betexplorer.com/soccer/world/world-cup-2022/']


# https://www.betexplorer.com/soccer/world/friendly-international-2013/results/?month=2013-02
urls = []
games_bets_rows = []
for wc in tqdm(wcs):
    url = wc
    #print(wc)
    try:
        page = requests.get(url)

        content = page.content
        soup = BeautifulSoup(page.content, 'html.parser')
        tr_rows = soup.find_all('tr')


        ix = 0
        for row in tr_rows:
            #print(ix)
            ix=ix+1
            r = str(row)
            #print(r)

            if 'in-match' in r:
                logging.info(r)
                matchday = r.split('<td class="h-text-right h-text-no-wrap">')[1]

                temp = BeautifulSoup(r, 'html.parser')
                spans = temp.find_all('span')
                i=0
                for span in spans:
                    if i < 2:

                        s = str(span)
                        s = s.replace('<','').replace('>','').replace('strong','').replace('span','').replace('/','')
                        logging.info(f'      {s}')
                        if i == 0:
                            home_team=s
                        else:
                            away_team=s
                        i=i+1


                odds = temp.find_all('td', class_='table-main__odds')
                input_tag = temp.find("td", {"name":"City"})
                odds_list=[]
                valid_odds=False
                for odd in odds:
                    o = str(odd).split('data-odd=')
                    logging.info(o[1].split('"')[1])
                    odds_list.append(o[1].split('"')[1])
                    valid_odds=True

                games = temp.find_all('td', class_='h-text-center')
                for game in games:
                    gam= str(game)
                    #logging.info(gam)
                    # get the game + key
                    g = gam.split('href="')[1].split('">')[0]
                    g = g.split('/')
                    g = f"{g[1]}/{g[2]}/{g[3]}/{g[4]}-{g[5]}"
                    # get game score
                    s = gam.split('</a')[0]
                    s = s.split('/">')[1]
                    s = s.split(' ')[0]
                    if ':' in s:
                        logging.info(s)
                        correct_score=s
                        game_entry=g
                        game_url = url
                        urls.append([g,url,s])
                logging.info(matchday.split('<')[0])
                matchday = matchday.split('<')[0]

                if valid_odds:
                    games_bets_rows.append([game_entry, matchday, home_team, away_team, correct_score, odds_list[0], odds_list[1], odds_list[2], game_url ])


    except Exception as e:
        logging.info(f'no-game - {e}')
        #print('no Game')
        #print(e)
        #print(soup)

        
df = pd.DataFrame(games_bets_rows)
df.columns = ['oddsportal','matchday','home_team','away_team','correct_score','1N2_1','1N2_N','1N2_2','betexplorer']
df.drop_duplicates(inplace=True)
df.to_csv(f'data/urls_{timestampStr}.csv', index=False)
df.to_csv(f'data/oddsportal_urls_{timestampStr}.csv')

"""
for idx, r in tqdm(df.iterrows()):
    odp = f'https://www.oddsportal.com/{r.oddsportal}'
    page = requests.get(odp)

    content = page.content
    soup = BeautifulSoup(page.content, 'html.parser')
    p_class = soup.find_all('p', class_='result')
    #df.loc[idx,'result']=p_class
"""

print(f'Rows scrapped {df.shape}')
df.sample(8)

for idx, r in df.iterrows():
    d=r.matchday
    if len(d)<10:
        df.loc[idx,'matchday']=f'{d}2022'
df['matchdate'] = pd.to_datetime(df['matchday'],  format="%d.%m.%Y")
df['period']=df['matchdate'].dt.to_period('M')
df.drop_duplicates(['matchday','home_team','away_team'], inplace=True)
df.shape



for idx, r in tqdm(df.iterrows()):
    tags = r.oddsportal.split('/')
    df.loc[idx,'location']=tags[1]
    df.loc[idx,'season']=tags[2].split('-')[-1]
    l=tags[2].split('-')[:-1]
    df.loc[idx,'competition']='-'.join(l)
    # compute 1/N/2
    scores = r.correct_score
    score = scores.split(':')
    df.loc[idx,'home_score']=int(score[0])
    df.loc[idx,'away_score']=int(score[1])
    if score[0]>score[1]:
        df.loc[idx,'bet_won']='1'    
        df.loc[idx,'target']=0
        df.loc[idx,'odd_win']=r['1N2_1']    
        if r['1N2_1']<r['1N2_N'] and r['1N2_1']<r['1N2_2']:
            df.loc[idx,'best_odd_won']=True
        else:
            df.loc[idx,'best_odd_won']=False
    elif score[0]<score[1]:
        df.loc[idx,'bet_won']='2'
        df.loc[idx,'target']=2
        df.loc[idx,'odd_win']=r['1N2_2'] 
        if r['1N2_2']<r['1N2_N'] and r['1N2_2']<r['1N2_1']:
            df.loc[idx,'best_odd_won']=True
        else:
            df.loc[idx,'best_odd_won']=False

    else:
        df.loc[idx,'bet_won']='N'
        df.loc[idx,'target']=1       
        df.loc[idx,'odd_win']=r['1N2_N'] 
        if r['1N2_N']<r['1N2_1'] and r['1N2_N']<r['1N2_2']:
            df.loc[idx,'best_odd_won']=True
        else:
            df.loc[idx,'best_odd_won']=False

    
try:
    df.drop('matchdate', inplace=True)
    # Convert multiple columns

except:
    pass

df = df.astype({'1N2_1':'float','1N2_N':'float','1N2_2':'float','odd_win':'float','best_odd_won':'boolean'})    
df.drop_duplicates(inplace=True)
df.drop(['oddsportal','betexplorer','matchday'],axis=1).drop_duplicates().to_csv(f'data/20-years-odds.csv', index=False)

df.drop(['oddsportal','betexplorer','matchday'],axis=1).drop_duplicates().sample(8)

100%|██████████| 192/192 [06:32<00:00,  2.04s/it]


Rows scrapped (65960, 9)


65960it [01:05, 1010.56it/s]


,home_team,away_team,correct_score,1N2_1,1N2_N,1N2_2,matchdate,period,location,season,competition,home_score,away_score,bet_won,target,odd_win,best_odd_won
31524,Portogruaro,Sassuolo,1:0,4.25,3.08,1.89,2010-09-05,2010-09,italy,2011,serie-b-2010,1.0,0.0,1,0.0,4.25,False
16626,Atalanta,Parma,1:0,2.46,2.94,2.75,2005-03-13,2005-03,italy,2005,serie-a-2004,1.0,0.0,1,0.0,2.46,True
43750,Rotherham,Sheffield Wed,1:2,3.01,3.29,2.40,2015-10-23,2015-10,england,2016,championship-2015,1.0,2.0,2,2.0,2.40,True
42257,Perugia,Pro Vercelli,3:2,1.93,3.17,4.21,2015-03-14,2015-03,italy,2015,serie-b-2014,3.0,2.0,1,0.0,1.93,True
50903,AFC Wimbledon,Southend,2:0,2.47,3.23,2.90,2018-01-01,2018-01,england,2018,league-one-2017,2.0,0.0,1,0.0,2.47,True
9852,Notts Co,Huddersfield,3:2,1.87,3.31,3.59,2003-03-29,2003-03,england,2003,league-one-2002,3.0,2.0,1,0.0,1.87,True
40976,Blackpool,Watford,0:1,3.91,3.41,1.95,2014-09-16,2014-09,england,2015,championship-2014,0.0,1.0,2,2.0,1.95,True
39107,Freiburg,Hoffenheim,1:1,2.91,3.46,2.36,2014-02-08,2014-02,germany,2014,bundesliga-2013,1.0,1.0,N,1.0,3.46,False
